# CH07 : Using Generators

#### 1.제너레이터 만들기
: 파이썬에서 고성능이면서도 메모리를 적게 사요한느 반복을 위한 방법으로 사용  
(2001 PEP-255t소개)

##### 1-1. 제너레이터 개요

In [34]:
class PurchasesStats:
    def __init__(self, purchases):
        self.purchases = iter(purchases)
        self.min_price: float = None
        self.max_price: float = None
        self._total_purchases_price: float = 0.0
        self._total_purchases = 0
        self._initialize()

    def _initialize(self):
        try:
            first_value = next(self.purchases)
        except StopIteration:
            raise ValueError("no values provided")

        self.min_price = self.max_price = first_value
        self._update_avg(first_value)

    def process(self):
        for purchase_value in self.purchases:
            self._update_min(purchase_value)
            self._update_max(purchase_value)
            self._update_avg(purchase_value)
        return self

    def _update_min(self, new_value: float):
        if new_value < self.min_price:
            self.min_price = new_value

    def _update_max(self, new_value: float):
        if new_value > self.max_price:
            self.max_price = new_value

    @property
    def avg_price(self):
        return self._total_purchases_price / self._total_purchases

    def _update_avg(self, new_value: float):
        self._total_purchases_price += new_value
        self._total_purchases += 1

    def __str__(self):
        return (
            f"{self.__class__.__name__}({self.min_price}, "
            f"{self.max_price}, {self.avg_price})"
        )

In [35]:
def _load_purchases(filename):
    purchases = []
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            purchases.append(float(price_raw))

    return purchases

In [36]:
def load_purchases(filename):
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            yield float(price_raw)

In [37]:
load_purchases("file")

<generator object load_purchases at 0x10c75a350>

#### Generators expression

In [38]:
sum(x**2 for x in range(10))

285

#### 2. 이상적인 반복

##### 2-1. 관용적인 반복코드

In [39]:
list(enumerate("abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

In [40]:
class NumberSequence:
    def __init__(self, start=0):
        self.current = start

    def next(self):
        current = self.current
        self.current += 1
        return current

In [41]:
seq = NumberSequence()
seq.next()

0

In [42]:
seq.next()

1

In [43]:
seq2 = NumberSequence(10)
seq2.next()

10

In [44]:
seq2.next()

11

In [45]:
list(zip(NumberSequence(), "abcdef"))

TypeError: 'NumberSequence' object is not iterable

그러나 이 코드로 enumerate() 함수를 사용하도록 재작성할 수는 없다.  
왜냐하면 파이썬의 for 루프를 사용하기 위한 인터페이스를 지원하지 않기 때문이다.

In [46]:
""" __iter__() 매직 메서드를 구현하여 해결"""

class SequenceOfNumbers:
    def __init__(self, start=0):
        self.current = start

    def __next__(self):
        current = self.current
        self.current += 1
        return current

    def __iter__(self):
        return self

In [47]:
list(zip(SequenceOfNumbers(), "abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

In [48]:
seq = SequenceOfNumbers(100)
next(seq)

100

In [49]:
next(seq)

101

##### next() function

In [50]:
word = iter("Hello")
next(word)

'H'

In [51]:
next(word)

'e'

In [52]:
next(word, "default value")

'l'

##### using Generators

In [53]:
def sequence(start=0):
    while True:
        yield start
        start += 1

In [54]:
seq = sequence(10)
list(zip(sequence(), "abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

##### itertools

In [55]:
from itertools import islice

purchases = islice(filter(lambda p: p > 1000.0, purchases), 10)
stats = PurchasesStats(purchases).process()

NameError: name 'purchases' is not defined

##### 이터레이터를 사용한 코드 간소화

- 여러 번 반복하기

In [ ]:
def process_purchases(purchases):
    min_, max_, avg = itertools.tee(purchases, 3)
    return min(min_), max(max_), median(avg)

TIP:반복을여러번해야 되는경우 itertool.tee를사용한다.

- 중첩루프 (not good)

In [ ]:
""" bad code"""
def search_nested_bad(array, desired_value):
    coords = None
    for i, row in enumerate(array):
        for j, cell in enumerate(row):
            if cell == desired_value:
                coords = (i, j)
                break

        if coords is not None:
            break

    if coords is None:
        raise ValueError(f"{desired_value} not found")

    logger.info("value %r found at [%i, %i]", desired_value, *coords)
    return coords

In [ ]:
def _iterate_array2d(array2d):
    for i, row in enumerate(array2d):
        for j, cell in enumerate(row):
            yield (i, j), cell


def search_nested(array, desired_value):
    try:
        coord = next(
            coord
            for (coord, cell) in _iterate_array2d(array)
            if cell == desired_value
        )
    except StopIteration:
        raise ValueError("{desired_value} not found")

    logger.debug("value %r found at [%i, %i]", desired_value, *coord)
    return coord

| 최대한 중첩 루프를 제거하고 추상화하여 반복을 단순화한다.

#### 2-2. 파이썬의 이터레이터 패턴

제너레이터는 이터러블 객체의 특병ㄹ한 경우

이터레이터 ex) _ _iter_ _() / _ _next_ _()

##### 이터레이션 인터페이스

: 이터러블, 반복을 지원하는 객체

_ _iter_ _ 매직 메서들ㄹ 통해 이터레이터 반환  

_ _next_ _ 매직 메서드를 통해 반복 로직 구현


In [2]:
class SequenceIterator:
    def __init__(self, start=0, step=1):
        self.current = start
        self.step = step
    
    def __next__(self):
        value = self.current
        self.current += self.step
        return value

In [3]:
si = SequenceIterator(1,2)
next(si)

1

In [4]:
next(si)

3

In [5]:
next(si)

5

In [6]:
for _ in SequenceIterator(): pass
# __iter__() 메서드를 구현하지 않았기 떄문

TypeError: 'SequenceIterator' object is not iterable

##### 이터러블이 가능한 시퀀스 객체

In [8]:
"""Clean Code in Python - Chapter 7: Using Generators

> The Interface for Iteration: sequences

"""

import logging

logger = logging.getLogger(__name__)


class SequenceWrapper:
    def __init__(self, original_sequence):
        self.seq = original_sequence

    def __getitem__(self, item):
        value = self.seq[item]
        logger.debug("%s getting %s", self.__class__.__name__, item)
        return value

    def __len__(self):
        return len(self.seq)


class MappedRange:
    """Apply a transformation to a range of numbers."""

    def __init__(self, transformation, start, end):
        self._transformation = transformation
        self._wrapped = range(start, end)

    def __getitem__(self, index):
        value = self._wrapped.__getitem__(index)
        result = self._transformation(value)
        logger.debug("Index %d: %s", index, result)
        return result

    def __len__(self):
        return len(self._wrapped)


In [11]:
mr = MappedRange(abs, -10, 5)
print(mr[0])

10


In [12]:
print(mr[-1])

4


In [13]:
print(list(mr))

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4]


#### 3. coroutine

- 코루틴의 인터페이스
    - close()
        - generator에서 GeneratorExit 예외가 발생하며, 이 예외를 따로 처리하지 않으면  
        제너레이터 더 이상 값을 생성하지 않으며 반복이 중지
    - throw(ex_type[, ex_value[, ex_traceback]])
        - 현재 제너레이터가 중단된 위치에서 예외를 던짐
        
    - send(value)
        - send() 메서드를 사용했다는 것은 yield 키워드가 할당 구문의 오른쪽에 나오게 되고 인자 값을 받아서 다른 곳에 할당할 수 있음을 뜻함
        - 코루틴에서 send() 메서드를 호출하려면 항상 next()를 먼저 호출.
        - next()를 자동 호출해주는 @prepare_coroutine을 사용하면 편함

In [17]:
"""Clean Code in Python - Chapter 7: Using Generators

> Methods of the Generators Interface.

"""
import time

# from log import logger
import logging

class DBHandler:
    """Simulate reading from the database by pages."""
    def __init__(self, db):
        self.db = db
        self.is_closed = False

    def read_n_records(self, limit):
        return [(i, f"row {i}") for i in range(limit)]

    def close(self):
        logger.debug("closing connection to database %r", self.db)
        self.is_closed = True


def stream_db_records(db_handler):
    """Example of .close()

    >>> streamer = stream_db_records(DBHandler("testdb"))  # doctest: +ELLIPSIS
    >>> len(next(streamer))
    10

    >>> len(next(streamer))
    10
    """
    try:
        while True:
            yield db_handler.read_n_records(10)
            time.sleep(.1)
    except GeneratorExit:
        db_handler.close()


class CustomException(Exception):
    """An exception of the domain model."""


def stream_data(db_handler):
    """Test the ``.throw()`` method.

    >>> streamer = stream_data(DBHandler("testdb"))
    >>> len(next(streamer))
    10
    """
    while True:
        try:
            yield db_handler.read_n_records(10)
        except CustomException as e:
            logger.info("controlled error %r, continuing", e)
        except Exception as e:
            logger.info("unhandled error %r, stopping", e)
            db_handler.close()
            break


In [19]:
streamer = stream_data(DBHandler("testdb"))
next(streamer)

[(0, 'row 0'),
 (1, 'row 1'),
 (2, 'row 2'),
 (3, 'row 3'),
 (4, 'row 4'),
 (5, 'row 5'),
 (6, 'row 6'),
 (7, 'row 7'),
 (8, 'row 8'),
 (9, 'row 9')]

In [20]:
next(streamer)

[(0, 'row 0'),
 (1, 'row 1'),
 (2, 'row 2'),
 (3, 'row 3'),
 (4, 'row 4'),
 (5, 'row 5'),
 (6, 'row 6'),
 (7, 'row 7'),
 (8, 'row 8'),
 (9, 'row 9')]

#### 3-2. Coroutine 고급 주제

In [21]:
""" Stop exception을 활용한 리턴 방법(실제로 쓰이진 않음) """
def generator():
    yield 1
    yield 2
    return 3

gen = generator()

while(True):
    try:
        next(gen)
    except StopIteration as e:
        print(">>> returned value", e.value)
        break

>>> returned value 3


In [22]:
""" 간단한 yield from 사용 예""" 
def chain(*iterables):
    for it in iterables:
        for values in it:
            yield value


In [25]:
def chain(*iterables):
    for it in iterables:
        yield from it # 

In [26]:
list(chain("hello", ["world"], ("tuple", "of","value")))

['h', 'e', 'l', 'l', 'o', 'world', 'tuple', 'of', 'value']

##### 서브 제너레이터에서 반환된 값 구하기

In [27]:
def sequence(name, start, end):
    logger.info("%s started at %i", name, start)
    yield from range(start, end)
    logger.info("%s finished at %i", name, end)
    return end

def main():
    step1 = yield from sequence("first", 0, 5)
    step2 = yield from sequence("second", step1, 10)
    return step1 + step2

g = main()

while(True): # yield from을 처음부터 쓸 수는 없음
    try:
        next(g)
    except StopIteration as e:
        print(">>> returned value", e.value) # 리턴값을 반환받을 수 있음
        break

>>> returned value 15


##### 서브 제너레이터와 데이터 송수신하기

In [28]:
class CustomException(Exception):
    """A type of exception that is under control."""


def sequence(name, start, end):
    value = start
    logger.info("%s started at %i", name, value)
    while value < end:
        try:
            received = yield value
            logger.info("%s received %r", name, received)
            value += 1
        except CustomException as e:
            logger.info("%s is handling %s", name, e)
            received = yield "OK"
    return end


def main():
    step1 = yield from sequence("first", 0, 5)
    step2 = yield from sequence("second", step1, 10)

if __name__ == "__main__":
    print(list(main()))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
